In [1]:
!pip install requests-cache

     |████████████████████████████████| 44 kB 857 kB/s 
     |████████████████████████████████| 138 kB 9.9 MB/s 
     |████████████████████████████████| 127 kB 37.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
import requests
import numpy as np
import pandas as pd
import requests_cache
import lxml.html as html_parser
import re

In [34]:
MAX_PAGE = 333

In [35]:
## Get all the urls for all the listed used vehicles on truecar.com
def urls_scraping(base_url = 'https://www.truecar.com/new-cars-for-sale/listings/'):
    urls = []
    pages = []
    for i in range(1, MAX_PAGE+1):
        pages.append(base_url + '?page=' + str(i))
    for page in pages:
        try:
            response = requests.get(page)
            response.raise_for_status()
        except:
            break
        root = html_parser.fromstring(response.content)
        url = ['https://www.truecar.com' + link for link in root.xpath('//div[@data-test="newOptionVehicleCard"]/a/@href')]
        urls += url
    
    return urls

In [36]:
def page_scraping(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except:
        return
    root = html_parser.fromstring(response.content)
    
    # extract vehicle year, make and model information
    year_make_model = root.xpath('//h1[contains(@class,"heading-base")]/div[@class="heading-2"]/text()')[0].strip()
    year = year_make_model.split()[0]
    make_model = year_make_model.split()[1:]
    make = make_model[0]

    model = ''
    i = 1

    while i < len(make_model):
      model += make_model[i] + ' '
      i += 1
    
    model = model.strip()

    sub_model = root.xpath('//h1[contains(@class,"heading-base")]/div[@class="heading-base"]/text()')[0].strip()

    # location = root.xpath('//div[@class="d-flex align-items-center padding-top-1"]/p/text()')[0].strip()
    # city = location.split(',')[0].strip()
    # state = location.split(',')[1].strip()

    price = root.xpath('//div[@data-test="vdpPreProspectPrice"]/text()')[0].strip()

    # style = root.xpath('//div[@data-qa="Heading" and contains(., "Style")]/following-sibling::p/text()')[0].strip()
    exterior_color = root.xpath('//div[@data-qa="Heading" and contains(., "Exterior Color")]/following-sibling::p/text()')[0].strip()
    # interior_color = root.xpath('//div[@data-qa="Heading" and contains(., "Interior Color")]/following-sibling::p/text()')[0].strip()
    engine = root.xpath('//div[@data-qa="Heading" and contains(., "Engine")]/following-sibling::p/text()')[0].strip()
    drive_type = root.xpath('//div[@data-qa="Heading" and contains(., "Drive Type")]/following-sibling::p/text()')[0].strip()
    fuel_type = root.xpath('//div[@data-qa="Heading" and contains(., "Fuel Type")]/following-sibling::p/text()')[0].strip()
    transmission = root.xpath('//div[@data-qa="Heading" and contains(., "Transmission")]/following-sibling::p/text()')[0].strip()

    
    

    return pd.Series({'year':year, 'make':make, 'model':model, 'sub_model':sub_model,
                      'price':price, 'exterior_color':exterior_color,  
                      'engine':engine, 'drive_type':drive_type, 'fuel_type':fuel_type,'transmission':transmission })

In [37]:
def scraping(urls):
    scraping_data = [page_scraping(url) for url in urls]
    return pd.concat(scraping_data, axis=1).T

In [38]:
%%time
urls=urls_scraping() # extract all vehicle urls from allowed 333 pages.
urls[:10]

CPU times: user 19.7 s, sys: 1.58 s, total: 21.2 s
Wall time: 29min 44s


In [40]:
print(len(urls))

9990


In [41]:
data = scraping(urls)

In [42]:
data.shape

(9958, 10)

In [43]:
data.tail(5)


,year,make,model,sub_model,price,exterior_color,engine,drive_type,fuel_type,transmission
9953,2021,Jeep,Grand Cherokee,L Limited 4WD,"$52,885",Bright White Clearcoat,3.6L V-6 Gas,4WD,Gas,Automatic
9954,2022,Ram,3500,Tradesman Crew Cab 8' Box 4WD,"$62,918",Hydro Blue Pearlcoat,6.7L V-6 Diesel Turbocharged,4WD,Diesel,Automatic
9955,2021,Ford,Ranger,XLT SuperCrew 5' Box 4WD,"$41,726",Cactus Gray,2.3L Inline-4 Gas Turbocharged,4WD,Gas,Automatic
9956,2022,Jeep,Compass,Trailhawk 4WD,"$37,715",Diamond Black Crystal Pearlcoat,2.4L Inline-4 Gas,4WD,Gas,Automatic
9957,2021,Jeep,Wrangler,Rubicon,"$50,357",Black Clearcoat,2.0L V-4 Gas Turbocharged,4WD,Gas,Automatic


In [44]:
data.to_csv('new-listing.csv', encoding='utf-8')